In [1]:
# CELL 1: Imports and Constants
import os
import glob
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from bs4 import BeautifulSoup
import re
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import emoji
import logging
import warnings
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn.functional import cosine_similarity as torch_cosine_similarity
from tqdm import tqdm

# Constants
EMBEDDING_DIM = 50
CONTEXT_WINDOW = 2
BATCH_SIZE = 32
NUM_EPOCHS = 20


# CELL 2: Logging Setup
def setup_logging(output_dir):
    """Configure logging to both file and console"""
    warnings.filterwarnings("ignore")
    logger = logging.getLogger('word2vec')
    logger.setLevel(logging.INFO)
    logger.handlers = []
    formatter = logging.Formatter('%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    
    log_file = os.path.join(output_dir, 'word2vec.log')
    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)
    
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    logger.info(f"Log file created at: {log_file}")
    return logger



# CELL 3: Data Processing Functions
def load_data(subset='train'):
    """Load data from positive and negative folders"""
    logger = logging.getLogger('word2vec')
    texts = []
    for label in ['positive', 'negative']:
        folder_path = os.path.join(subset, label)
        logger.info(f"Loading from: {folder_path}")
        
        if not os.path.exists(folder_path):
            raise Exception(f"Directory not found: {folder_path}")
            
        file_pattern = os.path.join(folder_path, '*.txt')
        files = glob.glob(file_pattern)
        
        if not files:
            logger.warning(f"No text files found in {folder_path}")
            
        for file_path in files:
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    texts.append(f.read().strip())
            except Exception as e:
                logger.error(f"Error reading file {file_path}: {str(e)}")
    
    logger.info(f"Loaded {len(texts)} texts from {subset} set")
    return texts

def preprocess_text(text):
    """Preprocess text with specific handling for cases and punctuation"""
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = emoji.demojize(text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    tokens = []
    for word in text.split():
        word = re.sub(r'@', '', word)
        punctuation = '!?.,;:()[]{}""\'\'``'
        
        if any(p in word for p in punctuation):
            parts = re.findall(r'\w+|[' + punctuation + ']', word)
            for part in parts:
                if part in punctuation:
                    tokens.extend([part] * len(part))
                else:
                    if not part.isupper():
                        part = part.lower()
                    tokens.append(part)
        else:
            if not word.isupper():
                word = word.lower()
            tokens.append(word)
    
    return tokens

def build_vocab(texts):
    """Build vocabulary from texts with word counts"""
    word_counts = defaultdict(int)
    for text in texts:
        tokens = preprocess_text(text)
        for token in tokens:
            word_counts[token] += 1
    
    vocab = {word: idx for idx, (word, _) in enumerate(word_counts.items())}
    idx2word = {idx: word for word, idx in vocab.items()}
    
    return vocab, idx2word, word_counts

def generate_pairs(tokens, window_size, vocab):
    """Generate positive and negative pairs for training"""
    pairs = []
    labels = []
    
    for i in range(len(tokens)):
        target = tokens[i]
        if target not in vocab:
            continue
            
        left = max(0, i - window_size)
        right = min(len(tokens), i + window_size + 1)
        
        for j in range(left, right):
            if i != j and tokens[j] in vocab:
                pairs.append((vocab[target], vocab[tokens[j]]))
                labels.append(1)
                
                for _ in range(2):
                    neg_idx = random.choice(list(vocab.values()))
                    while neg_idx == vocab[target] or neg_idx == vocab[tokens[j]]:
                        neg_idx = random.choice(list(vocab.values()))
                    pairs.append((vocab[target], neg_idx))
                    labels.append(0)
    
    return pairs, labels


# CELL 4: Dataset Class
class Word2VecDataset(Dataset):
    def __init__(self, pairs, labels):
        self.pairs = torch.LongTensor(pairs)
        self.labels = torch.FloatTensor(labels)
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        return self.pairs[idx], self.labels[idx]
    
    
# CELL 5: Model Classes
class SimpleWord2Vec_LogiR(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SimpleWord2Vec_LogiR, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.embeddings_context = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim * 2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inputs):
        target, context = inputs[:, 0], inputs[:, 1]
        target_emb = self.embeddings(target)
        context_emb = self.embeddings_context(context)
        concat_emb = torch.cat([target_emb, context_emb], dim=1)
        out = self.sigmoid(self.linear(concat_emb))
        return out.squeeze()

class SimpleWord2Vec_FFNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, node_size=64):
        super(SimpleWord2Vec_FFNN, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.embeddings_context = nn.Embedding(vocab_size, embedding_dim)
        
        self.linear1 = nn.Linear(embedding_dim * 2, node_size)
        self.linear2 = nn.Linear(node_size, node_size)
        self.linear3 = nn.Linear(node_size, 1)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inputs):
        target, context = inputs[:, 0], inputs[:, 1]
        target_emb = self.embeddings(target)
        context_emb = self.embeddings_context(context)
        
        concat_emb = torch.cat([target_emb, context_emb], dim=1)
        hidden1 = self.relu(self.linear1(concat_emb))
        hidden2 = self.relu(self.linear2(hidden1))
        out = self.sigmoid(self.linear3(hidden2))
        return out.squeeze()
    
    
# Improved training function with consistent logging and progress tracking
def train_model(model, train_loader, lr=0.01):
    """Train the model with consistent progress monitoring"""
    logger = logging.getLogger('word2vec')
    loss_function = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    losses = []
    total_batches = len(train_loader)
    logger.info(f"Starting training with {total_batches} total batches")
    
    try:
        for epoch in range(NUM_EPOCHS):
            epoch_loss = 0
            batch_count = 0
            
            # Use tqdm for progress tracking
            for batch_inputs, batch_labels in tqdm(train_loader, desc=f"Epoch {epoch}"):
                optimizer.zero_grad()
                outputs = model(batch_inputs)
                loss = loss_function(outputs, batch_labels)
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item()
                batch_count += 1
            
            avg_batch_loss = epoch_loss / batch_count
            losses.append(epoch_loss)
            
            # Consistent logging format
            logger.info(f"Epoch {epoch} completed:")
            logger.info(f"  Total Loss: {epoch_loss:.4f}")
            logger.info(f"  Average batch loss: {avg_batch_loss:.4f}")
            logger.info(f"  Number of batches: {batch_count}")
            logger.info(f"  Learning rate: {lr}")
            
            # Optional: Save checkpoint after each epoch
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
            }
            torch.save(checkpoint, os.path.join('results', f'checkpoint_epoch_{epoch}.pt'))
            
    except Exception as e:
        logger.error(f"Error during training: {str(e)}")
        # Save emergency checkpoint
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': losses[-1] if losses else None,
        }
        torch.save(checkpoint, os.path.join('results', 'emergency_checkpoint.pt'))
        raise e
    
    return model, losses

def visualize_embeddings(model, idx2word, num_words=30):
    """Visualize word embeddings using t-SNE"""
    logger = logging.getLogger('word2vec')
    try:
        logger.info("Starting visualization process...")
        embeddings = model.embeddings.weight.data.numpy()
        
        words = list(idx2word.values())[:num_words]
        word_embeddings = embeddings[:num_words]
        
        perplexity = min(30, len(words) - 1)
        tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity, n_iter=250)
        reduced_embeddings = tsne.fit_transform(word_embeddings)
        
        plt.clf()
        for i, word in enumerate(words):
            x, y = reduced_embeddings[i]
            plt.scatter(x, y, c='blue', alpha=0.5)
            plt.annotate(word, (x, y), fontsize=8, alpha=0.75)
        
        return True
    except Exception as e:
        logger.error(f"Error in visualization: {str(e)}")
        return False

def plot_losses(logir_losses, ffnn_losses, output_dir):
    """Plot training losses for both models"""
    logger = logging.getLogger('word2vec')
    plt.figure(figsize=(10, 6))
    epochs = range(len(logir_losses))
    
    # Plot both losses on the same graph
    plt.plot(epochs, logir_losses, 'b-', label='Logistic Regression')
    plt.plot(epochs, ffnn_losses, 'r-', label='FFNN')
    
    # Add labels and title
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Time')
    plt.legend()
    plt.grid(True)
    
    # Save the plot
    save_path = os.path.join(output_dir, 'loss_charts.png')
    plt.savefig(save_path)
    plt.close()
    logger.info(f"Loss charts saved to: {save_path}")
    return save_path
    

# CELL 7 (continued): Evaluation Functions
    with torch.no_grad():
        for i in range(0, len(test_pairs), batch_size):
            batch_pairs = test_pairs[i:i + batch_size]
            target_emb = embeddings[batch_pairs[:, 0]]
            context_emb = embeddings[batch_pairs[:, 1]]
            sim = torch_cosine_similarity(target_emb, context_emb, dim=1)
            similarities.extend(sim.cpu().numpy())
    
    similarities = np.array(similarities)
    labels = np.array(labels[:len(similarities)])
    
    pos_sim = similarities[labels == 1]
    neg_sim = similarities[labels == 0]
    
    return {
        'avg_positive_similarity': np.mean(pos_sim),
        'avg_negative_similarity': np.mean(neg_sim),
        'num_positive_pairs': len(pos_sim),
        'num_negative_pairs': len(neg_sim)
    }


# CELL 8: Embedding Comparison Function
def compare_embeddings(logir_model, ffnn_model, vocab, idx2word, output_dir):
    """Compare embeddings from both models"""
    logger = logging.getLogger('word2vec')
    logger.info("\nComparing embeddings between models...")
    
    logir_embeddings = logir_model.embeddings.weight.data.numpy()
    ffnn_embeddings = ffnn_model.embeddings.weight.data.numpy()
    
    similarities = []
    for i in range(len(vocab)):
        sim = cosine_similarity(
            logir_embeddings[i].reshape(1, -1),
            ffnn_embeddings[i].reshape(1, -1)
        )[0][0]
        similarities.append(sim)
    
    sorted_idx = np.argsort(similarities)
    most_similar = [(idx2word[i], similarities[i]) for i in sorted_idx[-5:]]
    least_similar = [(idx2word[i], similarities[i]) for i in sorted_idx[:5]]
    
    return {
        'average_similarity': np.mean(similarities),
        'std_similarity': np.std(similarities),
        'most_similar': most_similar,
        'least_similar': least_similar
    }

# CELL 9: Report Generation Function
def save_comprehensive_report(logir_eval, ffnn_eval, embedding_comparison, output_dir):
    """Save comprehensive evaluation report"""
    logger = logging.getLogger('word2vec')
    report_path = os.path.join(output_dir, 'evaluation_report.txt')
    
    with open(report_path, 'w') as f:
        f.write("=" * 50 + "\n")
        f.write("Word2Vec Models Evaluation Report\n")
        f.write("=" * 50 + "\n\n")
        
        # Model evaluations
        f.write("1. Model Evaluations\n")
        f.write("-" * 20 + "\n\n")
        
        # LogiR results
        f.write("Logistic Regression Model:\n")
        f.write(f"Average Positive Similarity: {logir_eval['avg_positive_similarity']:.4f}\n")
        f.write(f"Average Negative Similarity: {logir_eval['avg_negative_similarity']:.4f}\n")
        f.write(f"Number of Positive Pairs: {logir_eval['num_positive_pairs']}\n")
        f.write(f"Number of Negative Pairs: {logir_eval['num_negative_pairs']}\n\n")
        
        # FFNN results
        f.write("FFNN Model:\n")
        f.write(f"Average Positive Similarity: {ffnn_eval['avg_positive_similarity']:.4f}\n")
        f.write(f"Average Negative Similarity: {ffnn_eval['avg_negative_similarity']:.4f}\n")
        f.write(f"Number of Positive Pairs: {ffnn_eval['num_positive_pairs']}\n")
        f.write(f"Number of Negative Pairs: {ffnn_eval['num_negative_pairs']}\n\n")
        
        # Embedding comparison
        f.write("2. Embedding Comparison\n")
        f.write("-" * 20 + "\n\n")
        f.write(f"Average Similarity between Models: {embedding_comparison['average_similarity']:.4f}\n")
        f.write(f"Standard Deviation: {embedding_comparison['std_similarity']:.4f}\n\n")
        
        f.write("Most Similar Words between Models:\n")
        for word, sim in embedding_comparison['most_similar']:
            f.write(f"{word}: {sim:.4f}\n")
        f.write("\n")
        
        f.write("Least Similar Words between Models:\n")
        for word, sim in embedding_comparison['least_similar']:
            f.write(f"{word}: {sim:.4f}\n")
        f.write("\n")
        
        # Analysis summary
        f.write("3. Analysis Summary\n")
        f.write("-" * 20 + "\n\n")
        
        logir_diff = logir_eval['avg_positive_similarity'] - logir_eval['avg_negative_similarity']
        ffnn_diff = ffnn_eval['avg_positive_similarity'] - ffnn_eval['avg_negative_similarity']
        
        f.write("Model Performance Comparison:\n")
        f.write(f"LogiR Positive-Negative Difference: {logir_diff:.4f}\n")
        f.write(f"FFNN Positive-Negative Difference: {ffnn_diff:.4f}\n\n")
        
        better_model = "Logistic Regression" if logir_diff > ffnn_diff else "FFNN"
        f.write(f"The {better_model} model shows better discrimination between positive and negative pairs.\n\n")
        
        # Embedding similarity analysis
        f.write("Embedding Similarity Analysis:\n")
        if embedding_comparison['average_similarity'] > 0.7:
            f.write("The models learned very similar embeddings, suggesting they captured similar semantic relationships.\n")
        elif embedding_comparison['average_similarity'] > 0.4:
            f.write("The models learned moderately similar embeddings, with some differences in their semantic representations.\n")
        else:
            f.write("The models learned quite different embeddings, suggesting they captured different aspects of the semantic relationships.\n")
    
    logger.info(f"Comprehensive evaluation report saved to: {report_path}")
    

# Detailed embedding comparison functions
def analyze_embeddings(model_name, embeddings, idx2word, vocab_size=1000):
    """Analyze embeddings of a single model"""
    # Get most frequent words (first N words in vocabulary)
    frequent_words = [idx2word[i] for i in range(min(vocab_size, len(idx2word)))]
    frequent_embeddings = embeddings[:vocab_size]
    
    # Calculate cosine similarity matrix for frequent words
    similarity_matrix = cosine_similarity(frequent_embeddings)
    
    # Find most similar word pairs
    most_similar_pairs = []
    for i in range(len(frequent_words)):
        for j in range(i + 1, len(frequent_words)):
            similarity = similarity_matrix[i, j]
            most_similar_pairs.append((frequent_words[i], frequent_words[j], similarity))
    
    # Sort by similarity
    most_similar_pairs.sort(key=lambda x: x[2], reverse=True)
    
    return {
        'model_name': model_name,
        'embedding_norm': np.linalg.norm(embeddings),
        'embedding_mean': np.mean(embeddings),
        'embedding_std': np.std(embeddings),
        'most_similar_pairs': most_similar_pairs[:10]  # Top 10 most similar pairs
    }

def compare_model_embeddings(logir_model, ffnn_model, vocab, idx2word, output_dir):
    """Compare embeddings from both models and generate detailed report"""
    logger = logging.getLogger('word2vec')
    logger.info("\nComparing embeddings between models...")
    
    # Extract embeddings
    logir_embeddings = logir_model.embeddings.weight.data.numpy()
    ffnn_embeddings = ffnn_model.embeddings.weight.data.numpy()
    
    # Analyze each model's embeddings
    logir_analysis = analyze_embeddings("Logistic Regression", logir_embeddings, idx2word)
    ffnn_analysis = analyze_embeddings("FFNN", ffnn_embeddings, idx2word)
    
    # Calculate overall similarity between models
    # Sample a subset of words for efficiency
    sample_size = min(1000, len(vocab))
    sample_indices = np.random.choice(len(vocab), sample_size, replace=False)
    
    logir_sample = logir_embeddings[sample_indices]
    ffnn_sample = ffnn_embeddings[sample_indices]
    
    # Calculate cosine similarity between corresponding embeddings
    similarities = []
    for i in range(sample_size):
        sim = cosine_similarity(
            logir_sample[i].reshape(1, -1),
            ffnn_sample[i].reshape(1, -1)
        )[0][0]
        similarities.append(sim)
    
    # Generate comparison report
    report_path = os.path.join(output_dir, 'embedding_comparison_report.txt')
    with open(report_path, 'w') as f:
        f.write("=" * 80 + "\n")
        f.write("Word2Vec Models Embedding Comparison Report\n")
        f.write("=" * 80 + "\n\n")
        
        # Model Statistics
        f.write("1. Model Statistics\n")
        f.write("-" * 20 + "\n\n")
        
        for analysis in [logir_analysis, ffnn_analysis]:
            f.write(f"{analysis['model_name']} Model:\n")
            f.write(f"  Embedding Norm: {analysis['embedding_norm']:.4f}\n")
            f.write(f"  Mean: {analysis['embedding_mean']:.4f}\n")
            f.write(f"  Standard Deviation: {analysis['embedding_std']:.4f}\n\n")
            
            f.write("  Top 10 Most Similar Word Pairs:\n")
            for word1, word2, sim in analysis['most_similar_pairs']:
                f.write(f"    {word1:20} - {word2:20}: {sim:.4f}\n")
            f.write("\n")
        
        # Model Comparison
        f.write("2. Model Comparison\n")
        f.write("-" * 20 + "\n\n")
        
        avg_similarity = np.mean(similarities)
        std_similarity = np.std(similarities)
        
        f.write(f"Average Similarity between Models: {avg_similarity:.4f}\n")
        f.write(f"Similarity Standard Deviation: {std_similarity:.4f}\n\n")
        
        # Analysis Summary
        f.write("3. Analysis and Insights\n")
        f.write("-" * 20 + "\n\n")
        
        # Compare embedding distributions
        f.write("Embedding Distribution Analysis:\n")
        if abs(logir_analysis['embedding_mean'] - ffnn_analysis['embedding_mean']) < 0.1:
            f.write("- Both models learned similar average embedding values\n")
        else:
            f.write("- Models learned different average embedding values\n")
            
        if abs(logir_analysis['embedding_std'] - ffnn_analysis['embedding_std']) < 0.1:
            f.write("- Both models show similar embedding spread\n")
        else:
            f.write("- Models show different levels of embedding spread\n")
        
        # Analyze similarity patterns
        f.write("\nSimilarity Analysis:\n")
        if avg_similarity > 0.7:
            f.write("- Models learned very similar word representations\n")
        elif avg_similarity > 0.4:
            f.write("- Models learned moderately similar word representations\n")
        else:
            f.write("- Models learned quite different word representations\n")
            
        # Compare word relationships
        common_pairs = set([(w1, w2) for w1, w2, _ in logir_analysis['most_similar_pairs'][:5]]).intersection(
            set([(w1, w2) for w1, w2, _ in ffnn_analysis['most_similar_pairs'][:5]])
        )
        
        f.write(f"\nCommon Word Relationships:\n")
        f.write(f"- Found {len(common_pairs)} common word pairs in top 5 most similar pairs\n")
        
        # Conclusion
        f.write("\nConclusion:\n")
        if avg_similarity > 0.6:
            f.write("The two models have learned largely similar word embeddings, suggesting they've captured similar semantic relationships despite their different architectures.")
        elif avg_similarity > 0.3:
            f.write("The models have learned moderately different embeddings, indicating that the architectural differences led to somewhat different semantic representations.")
        else:
            f.write("The models have learned substantially different embeddings, suggesting that the architectural differences significantly impact how semantic relationships are captured.")
    
    logger.info(f"Embedding comparison report saved to: {report_path}")
    return {
        'logir_analysis': logir_analysis,
        'ffnn_analysis': ffnn_analysis,
        'average_similarity': avg_similarity,
        'similarity_std': std_similarity
    }


def evaluate_model(model, test_texts, vocab, idx2word, output_dir, model_name):
    """
    Evaluate model using cosine similarity on test data
    Returns dictionary with evaluation metrics
    """
    logger = logging.getLogger('word2vec')
    logger.info(f"\nEvaluating {model_name} model...")
    
    # Extract embeddings
    with torch.no_grad():
        embeddings = model.embeddings.weight.data
    
    # Generate test pairs
    test_pairs = []
    labels = []
    for text in test_texts:
        tokens = preprocess_text(text)
        pairs, pair_labels = generate_pairs(tokens, CONTEXT_WINDOW, vocab)
        test_pairs.extend(pairs)
        labels.extend(pair_labels)
    
    # Convert to tensors
    test_pairs = torch.LongTensor(test_pairs)
    
    # Calculate similarities
    similarities = []
    batch_size = 1000  # Process in batches to avoid memory issues
    
    with torch.no_grad():
        for i in range(0, len(test_pairs), batch_size):
            batch_pairs = test_pairs[i:i + batch_size]
            target_emb = embeddings[batch_pairs[:, 0]]
            context_emb = embeddings[batch_pairs[:, 1]]
            
            # Calculate cosine similarity
            sim = torch_cosine_similarity(target_emb, context_emb, dim=1)
            similarities.extend(sim.cpu().numpy())
    
    # Calculate metrics
    similarities = np.array(similarities)
    labels = np.array(labels[:len(similarities)])
    
    pos_sim = similarities[labels == 1]
    neg_sim = similarities[labels == 0]
    
    # Calculate average similarity for positive and negative pairs
    avg_pos_sim = np.mean(pos_sim)
    avg_neg_sim = np.mean(neg_sim)
    
    # Save evaluation results
    eval_results = {
        'avg_positive_similarity': avg_pos_sim,
        'avg_negative_similarity': avg_neg_sim,
        'num_positive_pairs': len(pos_sim),
        'num_negative_pairs': len(neg_sim),
        'positive_std': np.std(pos_sim),
        'negative_std': np.std(neg_sim)
    }
    
    # Save detailed results to file
    results_file = os.path.join(output_dir, f'{model_name.lower()}_evaluation.txt')
    with open(results_file, 'w') as f:
        f.write(f"{model_name} Model Evaluation Results\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Number of test pairs: {len(similarities)}\n")
        f.write(f"Positive pairs: {len(pos_sim)}\n")
        f.write(f"Negative pairs: {len(neg_sim)}\n\n")
        f.write(f"Average positive similarity: {avg_pos_sim:.4f} (±{np.std(pos_sim):.4f})\n")
        f.write(f"Average negative similarity: {avg_neg_sim:.4f} (±{np.std(neg_sim):.4f})\n")
        f.write(f"Discrimination (pos - neg): {avg_pos_sim - avg_neg_sim:.4f}\n")
    
    logger.info(f"Evaluation results saved to: {results_file}")
    return eval_results

if __name__ == "__main__":
    
        # CELL 10: Main Execution - Setup and Training
    # Create output directory
    output_dir = os.path.join('results')
    os.makedirs(output_dir, exist_ok=True)

    # Setup logging
    logger = setup_logging(output_dir)
    logger.info("Starting Word2Vec training process")

    # Load and preprocess data
    logger.info("Loading training data...")
    train_texts = load_data('train')

    # Build vocabulary
    logger.info("Building vocabulary...")
    vocab, idx2word, word_counts = build_vocab(train_texts)
    logger.info(f"Vocabulary size: {len(vocab)}")

    # Generate training pairs
    logger.info("Generating training pairs...")
    all_pairs = []
    all_labels = []
    for text in train_texts:
        tokens = preprocess_text(text)
        pairs, labels = generate_pairs(tokens, CONTEXT_WINDOW, vocab)
        all_pairs.extend(pairs)
        all_labels.extend(labels)

    logger.info(f"Generated {len(all_pairs)} training pairs")

    # Create dataset and dataloader
    dataset = Word2VecDataset(all_pairs, all_labels)
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    # Train LogiR model
    logger.info("\nTraining Logistic Regression model...")
    logir_model = SimpleWord2Vec_LogiR(len(vocab), EMBEDDING_DIM)
    trained_logir, logir_losses = train_model(logir_model, train_loader, lr=0.1)

    # Train FFNN model
    logger.info("\nTraining FFNN model...")
    ffnn_model = SimpleWord2Vec_FFNN(len(vocab), EMBEDDING_DIM, node_size=64)
    trained_ffnn, ffnn_losses = train_model(ffnn_model, train_loader, lr=0.1)

    # CELL 11: Main Execution - Evaluation and Visualization
    # Save models
    torch.save(trained_logir.state_dict(), os.path.join(output_dir, 'logir_model.pt'))
    torch.save(trained_ffnn.state_dict(), os.path.join(output_dir, 'ffnn_model.pt'))

    # Start evaluation
    logger.info("\nStarting model evaluation...")

    # Load test data
    test_texts = load_data('test')

    # Evaluate both models
    logir_eval = evaluate_model(trained_logir, test_texts, vocab, idx2word, output_dir, "LogiR")
    ffnn_eval = evaluate_model(trained_ffnn, test_texts, vocab, idx2word, output_dir, "FFNN")

    # Compare embeddings
    embedding_comparison = compare_embeddings(trained_logir, trained_ffnn, vocab, idx2word, output_dir)
    
    # Save comprehensive report
    save_comprehensive_report(logir_eval, ffnn_eval, embedding_comparison, output_dir)

    # Save losses to text file
    with open(os.path.join(output_dir, 'training_losses.txt'), 'w') as f:
        f.write("LogiR Losses:\n")
        f.write(",".join(map(str, logir_losses)))
        f.write("\n\nFFNN Losses:\n")
        f.write(",".join(map(str, ffnn_losses)))

    # Plot and save loss charts
    logger.info("\nGenerating loss charts...")
    plot_losses(logir_losses, ffnn_losses, output_dir)

    # Visualization
    logger.info("\nGenerating visualizations...")
    try:
        plt.figure(figsize=(15, 6))

        logger.info("Attempting LogiR visualization...")
        plt.subplot(1, 2, 1)
        success = visualize_embeddings(trained_logir, idx2word, num_words=30)
        if success:
            plt.title("Logistic Regression Model Embeddings")

        plt.tight_layout()
        save_path = os.path.join(output_dir, 'embeddings_visualization_Logit.png')
        plt.savefig(save_path)
        logger.info(f"Visualization saved to: {save_path}")

        logger.info("\nAttempting FFNN visualization...")
        plt.subplot(1, 2, 2)
        success = visualize_embeddings(trained_ffnn, idx2word, num_words=30)
        if success:
            plt.title("FFNN Model Embeddings")

        plt.tight_layout()
        save_path = os.path.join(output_dir, 'embeddings_visualization_FFNN.png')
        plt.savefig(save_path)
        logger.info(f"Visualization saved to: {save_path}")

    except Exception as e:
        logger.error(f"Error during visualization process: {str(e)}")
    finally:
        plt.close()

    logger.info("Training and evaluation process completed!")

2025-02-16 01:08:43 - Log file created at: results/word2vec.log
2025-02-16 01:08:43 - Starting Word2Vec training process
2025-02-16 01:08:43 - Loading training data...
2025-02-16 01:08:43 - Loading from: train/positive
2025-02-16 01:08:44 - Loading from: train/negative
2025-02-16 01:08:45 - Loaded 2904 texts from train set
2025-02-16 01:08:45 - Building vocabulary...
2025-02-16 01:08:45 - Vocabulary size: 6262
2025-02-16 01:08:45 - Generating training pairs...
2025-02-16 01:09:02 - Generated 568932 training pairs
2025-02-16 01:09:02 - 
Training Logistic Regression model...
2025-02-16 01:09:03 - Starting training with 17780 total batches
Epoch 0: 100%|██████████| 17780/17780 [00:29<00:00, 598.31it/s] 
2025-02-16 01:09:33 - Epoch 0 completed:
2025-02-16 01:09:33 -   Total Loss: 10406.3938
2025-02-16 01:09:33 -   Average batch loss: 0.5853
2025-02-16 01:09:33 -   Number of batches: 17780
2025-02-16 01:09:33 -   Learning rate: 0.1
Epoch 1: 100%|██████████| 17780/17780 [00:10<00:00, 1665.22